### Twilight Struggle

In [1]:
from twilight_ui import *

### Game Instance

In [2]:
u = UI()
u.run()

Initalising game..
Invalid command. Enter ? for help.
Invalid command. Enter ? for help.
Invalid command. Enter ? for help.
Invalid command. Enter ? for help.

The following commands are available:
?           Displays this help text.
m           Lists all possible moves, along with their respective IDs.
m <ID>      Makes a move.
m commit    Commits all moves made.
s           Displays the overall game state.
s ?         Shows help on game state queries.
c ?         Shows help on card information queries.

new         Start a new game.
quit        Exit the game.

